# IBM Applied Data Science Capstone Course by Coursera
## Week 5 Final Report
### Opening a New Restaurants in Vishakapatnam,INDIA 

* Build a dataframe of neighborhoods in Visakhapatnam, INDIA by web scraping the data from Wikipedia page
+ Get the geographical coordinates of the neighborhoods
+ Obtain the venue data for the neighborhoods from Foursquare API
+ Explore and cluster the neighborhoods
+ Select the best cluster to open a new restaurants

### 1. Import libraries

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup 
import json # library to handle JSON files
from urllib.request import urlopen
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values# to get coordinates
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed and imported!')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.1               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                       

### 2. Scrap data from Wikipedia page into a DataFrame

In [2]:
url="https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Visakhapatnam"
html= urlopen(url)

In [3]:
soup=BeautifulSoup(html.read(),'lxml')
type(soup)

bs4.BeautifulSoup

In [4]:
alllinks=soup.find_all('a')
lst=[]
for title in alllinks:
    lst.append(title.get_text())

In [5]:
lst
str_test=str(lst)
str_test

"['', 'Help', 'Jump to navigation', 'Jump to search', 'category', 'Visakhapatnam', '', 'Suburbs of Visakhapatnam', 'learn more', 'Abidnagar', 'Adarsh Nagar', 'Adavivaram', 'Aganampudi', 'Akkayyapalem', 'Akkireddypalem', 'Allipuram', 'Anakapalle', 'Anandapuram', 'Appikonda', 'Arilova', 'Asilmetta', 'Atchutapuram', 'Balayya Sastri Layout', 'Beach Road, Visakhapatnam', 'Bheemunipatnam', 'BHPV', 'CBM Compound', 'Chengal Rao Peta', 'Chinagantyada', 'Chinna Gadhili', 'Chinna Waltair', 'Chinnamushidiwada', 'Chintalagraharam', 'Daba Gardens', 'Daspalla Hills', 'Desapatrunipalem', 'Devada', 'Devipuram', 'Dondaparthy', 'Duvvada', 'Dwaraka Nagar', 'Gajuwaka', 'Gambhiram', 'Gandhigram, Visakhapatnam', 'Gangavaram, Visakhapatnam', 'Gidijala', 'Gnanapuram', 'Gopalapatnam', 'Gudilova', 'Hanumanthavaka', 'HB Colony', 'Isukathota', 'Jagadamba Centre', 'Jalari Peta', 'Jodugullapalem', 'Kailasapuram, Visakhapatnam', 'Kakani Nagar', 'Kancharapalem', 'Kapuluppada', 'Kirlampudi Layout', 'Kommadi', 'Kurmanna

In [7]:
neighbor= pd.DataFrame(lst,columns=['Neighbors'])

In [8]:
new=neighbor.drop(neighbor.index[:9])
neigh=new.drop(new.index[121:])
final=neigh.reset_index(drop=True)
df=pd.DataFrame(final)

In [9]:
df.shape

(121, 1)

### 3. Get the geographical coordinates

In [10]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_0172bd0fc0c84ed98838b821e05a86de = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='u0Nxq1p8EuhjLwAqRg46byag9QwGv5MUStGuYzkHm0Yc',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_0172bd0fc0c84ed98838b821e05a86de.get_object(Bucket='capstoneprojectweek5assignment-donotdelete-pr-ghtpbdgtrcihdt',Key='rio (1).xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_coor = pd.read_excel(body,index_col=0)
df_coor.head()


,Neighbors,Latitude,Longitude
0,"Abidnagar, Visakhapatnam",17.738865,83.299000
1,"Atchutapuram, Visakhapatnam",17.564214,82.978877
2,"Adarsh Nagar, Visakhapatnam",17.763627,83.332847
3,"Adavivaram, Visakhapatnam",17.776142,83.246586
4,"Aganampudi, Visakhapatnam",17.686435,83.137766


In [11]:
df_coor.shape

(121, 3)

### 4. Create a map ofVisakhapatnam with neighborhoods superimposed on top

In [12]:
map_vizag = folium.Map(location=[17.6868, 83.2185], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df_coor['Latitude'], df_coor['Longitude'], df_coor['Neighbors']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_vizag)  
    
map_vizag

### 5. Use the Foursquare API to explore the neighborhoods

In [13]:
# define Foursquare Credentials and Version
CLIENT_ID = 'TYWG0O3LF1WFQX5Z1ZM3J5NCKNUL3I42R2R2R1KUY4Y2IE5N' # your Foursquare ID
CLIENT_SECRET = '5IORPLE1EJDQTUGPRKBZKJG5L1NIYJTDO1X2QIYSPV1XJ3UI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TYWG0O3LF1WFQX5Z1ZM3J5NCKNUL3I42R2R2R1KUY4Y2IE5N
CLIENT_SECRET:5IORPLE1EJDQTUGPRKBZKJG5L1NIYJTDO1X2QIYSPV1XJ3UI


<b>Now, let's get the top 100 venues that are within a radius of 3300 meters.</b>

In [14]:
import requests
radius = 3300
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df_coor['Latitude'], df_coor['Longitude'], df_coor['Neighbors']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [15]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(3494, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"Abidnagar, Visakhapatnam",17.738865,83.299,Sangam Sarat Theatre,17.725508,83.302463,Indie Movie Theater
1,"Abidnagar, Visakhapatnam",17.738865,83.299,Pizza Hut,17.726650,83.305531,Pizza Place
2,"Abidnagar, Visakhapatnam",17.738865,83.299,Food Ex,17.722155,83.318422,Fast Food Restaurant
3,"Abidnagar, Visakhapatnam",17.738865,83.299,Sweet India,17.725087,83.316572,Snack Place
4,"Abidnagar, Visakhapatnam",17.738865,83.299,Cream & Fudge,17.719339,83.311927,Ice Cream Shop


<b>Let's check how many venues were returned for each neighorhood</b>

In [16]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
"Abidnagar, Visakhapatnam",63,63,63,63,63,63
"Adarsh Nagar, Visakhapatnam",19,19,19,19,19,19
"Adavivaram, Visakhapatnam",2,2,2,2,2,2
"Aganampudi, Visakhapatnam",4,4,4,4,4,4
"Akkayyapalem, Visakhapatnam",43,43,43,43,43,43
"Akkireddypalem, Visakhapatnam",5,5,5,5,5,5
"Allipuram, Visakhapatnam",46,46,46,46,46,46
"Anakapalle, Visakhapatnam",4,4,4,4,4,4
"Appikonda, Visakhapatnam",1,1,1,1,1,1


In [17]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 91 uniques categories.


<b>Let's find out how many unique categories can be curated from all the returned venues</b>

In [18]:
venues_df['VenueCategory'].unique()[:50]

array(['Indie Movie Theater', 'Pizza Place', 'Fast Food Restaurant',
       'Snack Place', 'Ice Cream Shop', 'Indian Restaurant', 'Café',
       'Restaurant', 'Hotel', 'Fabric Shop', 'Bookstore',
       'Italian Restaurant', 'Breakfast Spot',
       'Multicuisine Indian Restaurant', 'Coffee Shop', 'Multiplex',
       'Shopping Mall', 'Department Store', 'Dessert Shop', 'Food Court',
       'Train Station', 'Sandwich Place', 'Vegetarian / Vegan Restaurant',
       'Park', 'Volleyball Court', 'Bakery', 'Platform', 'Golf Course',
       'Dhaba', 'ATM', 'Mountain', 'Diner', 'Grocery Store', 'Beach',
       'Trail', 'Cafeteria', 'IT Services', 'Steakhouse',
       'Clothing Store', 'Airport', 'Juice Bar', 'Cricket Ground',
       'Harbor / Marina', 'Boat or Ferry', 'Pier', 'Athletics & Sports',
       'Smoke Shop', 'Science Museum', 'Resort', 'Lounge'], dtype=object)

In [19]:
"Restaurant" in venues_df['VenueCategory'].unique()

True

### 6. Analyze Each Neighborhood

In [20]:
vizag_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
vizag_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [vizag_onehot.columns[-1]] + list(vizag_onehot.columns[:-1])
vizag_onehot = vizag_onehot[fixed_columns]

print(vizag_onehot.shape)
vizag_onehot.head()

(3494, 92)


,Neighborhoods,ATM,Airport,Andhra Restaurant,Asian Restaurant,Athletics & Sports,Bakery,Bar,Basketball Stadium,Beach,...,Sporting Goods Shop,Steakhouse,Supermarket,Tea Room,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Volleyball Court
0,"Abidnagar, Visakhapatnam",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Abidnagar, Visakhapatnam",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Abidnagar, Visakhapatnam",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Abidnagar, Visakhapatnam",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Abidnagar, Visakhapatnam",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [21]:
vizag_grouped = vizag_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(vizag_grouped.shape)
vizag_grouped.head()

(114, 92)


,Neighborhoods,ATM,Airport,Andhra Restaurant,Asian Restaurant,Athletics & Sports,Bakery,Bar,Basketball Stadium,Beach,...,Sporting Goods Shop,Steakhouse,Supermarket,Tea Room,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Volleyball Court
0,"Abidnagar, Visakhapatnam",0.0,0.0,0.0,0.0,0.0,0.015873,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.031746,0.015873,0.0,0.015873
1,"Adarsh Nagar, Visakhapatnam",0.0,0.0,0.0,0.0,0.0,0.052632,0.0,0.0,0.052632,...,0.0,0.000000,0.0,0.0,0.0,0.052632,0.000000,0.000000,0.0,0.000000
2,"Adavivaram, Visakhapatnam",0.5,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000
3,"Aganampudi, Visakhapatnam",0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.750000,0.000000,0.0,0.000000
4,"Akkayyapalem, Visakhapatnam",0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.023256,0.0,0.0,0.0,0.000000,0.046512,0.000000,0.0,0.023256


In [22]:
len(vizag_grouped[vizag_grouped["Restaurant"] > 0])

54

<b> Create a new DataFrame for Shopping Mall data only</b>

In [23]:
vizag_mall = vizag_grouped[["Neighborhoods","Restaurant"]]

In [24]:
vizag_mall.head()

,Neighborhoods,Restaurant
0,"Abidnagar, Visakhapatnam",0.031746
1,"Adarsh Nagar, Visakhapatnam",0.000000
2,"Adavivaram, Visakhapatnam",0.000000
3,"Aganampudi, Visakhapatnam",0.000000
4,"Akkayyapalem, Visakhapatnam",0.069767


### 7. Cluster Neighborhoods

In [25]:
# set number of clusters
kclusters = 3

vizag_clustering = vizag_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(vizag_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 0, 0, 1, 0, 1, 2, 0, 0], dtype=int32)

In [26]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
vizag_merged = vizag_mall.copy()

# add clustering labels
vizag_merged["Cluster Labels"] = kmeans.labels_

In [27]:
vizag_merged.head()

,Neighborhoods,Restaurant,Cluster Labels
0,"Abidnagar, Visakhapatnam",0.031746,1
1,"Adarsh Nagar, Visakhapatnam",0.000000,0
2,"Adavivaram, Visakhapatnam",0.000000,0
3,"Aganampudi, Visakhapatnam",0.000000,0
4,"Akkayyapalem, Visakhapatnam",0.069767,1


In [28]:
# merge vizag_grouped with vizag_data to add latitude/longitude for each neighborhood
vizag_merged = vizag_merged.join(df_coor.set_index("Neighbors"), on="Neighborhoods")

print(vizag_merged.shape)
vizag_merged.head() # check the last columns!

(115, 5)


,Neighborhoods,Restaurant,Cluster Labels,Latitude,Longitude
0,"Abidnagar, Visakhapatnam",0.031746,1,17.738865,83.299000
1,"Adarsh Nagar, Visakhapatnam",0.000000,0,17.763627,83.332847
2,"Adavivaram, Visakhapatnam",0.000000,0,17.776142,83.246586
3,"Aganampudi, Visakhapatnam",0.000000,0,17.686435,83.137766
4,"Akkayyapalem, Visakhapatnam",0.069767,1,17.735277,83.299952


In [29]:
# sort the results by Cluster Labels
print(vizag_merged.shape)
vizag_merged.sort_values(["Cluster Labels"], inplace=True)
vizag_merged.head()

(115, 5)


,Neighborhoods,Restaurant,Cluster Labels,Latitude,Longitude
56,"Malkapuram, Visakhapatnam",0.000000,0,17.685467,83.243240
93,"Sheela Nagar, Visakhapatnam",0.000000,0,17.716136,83.196685
51,"MVP Colony, Visakhapatnam",0.026667,0,17.741312,83.334504
49,"Lankelapalem, Visakhapatnam",0.000000,0,17.663126,83.087136
48,"Kurmannapalem, Visakhapatnam",0.000000,0,17.690591,83.166442


<b>Finally, let's visualize the resulting clusters</b>

In [30]:
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[17.6868, 83.2185], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(vizag_merged['Latitude'], vizag_merged['Longitude'], vizag_merged['Neighborhoods'], vizag_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 8. Examine Clusters

In [31]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

<b>Cluster 0</b>

In [32]:
#cluster:0
vizag_merged.loc[vizag_merged['Cluster Labels'] == 0].head()

,Neighborhoods,Restaurant,Cluster Labels,Latitude,Longitude
56,"Malkapuram, Visakhapatnam",0.000000,0,17.685467,83.243240
93,"Sheela Nagar, Visakhapatnam",0.000000,0,17.716136,83.196685
51,"MVP Colony, Visakhapatnam",0.026667,0,17.741312,83.334504
49,"Lankelapalem, Visakhapatnam",0.000000,0,17.663126,83.087136
48,"Kurmannapalem, Visakhapatnam",0.000000,0,17.690591,83.166442


<b>Cluster 1</b>

In [33]:
#cluster:1
vizag_merged.loc[vizag_merged['Cluster Labels'] == 1].head()

,Neighborhoods,Restaurant,Cluster Labels,Latitude,Longitude
84,"Relli Veedhi, Visakhapatnam",0.060241,1,17.702535,83.303446
92,"Seethammapeta, Visakhapatnam",0.056604,1,17.731248,83.306905
85,"Resapuvanipalem, Visakhapatnam",0.043478,1,17.733626,83.316807
86,"Rushikonda, Visakhapatnam",0.066667,1,17.792547,83.384202
91,"Seethammadhara, Visakhapatnam",0.046154,1,17.742199,83.309109


<b>Cluster 2</b>

In [34]:
#cluster:2
vizag_merged.loc[vizag_merged['Cluster Labels'] == 2]

,Neighborhoods,Restaurant,Cluster Labels,Latitude,Longitude
54,"Madhurawada, Visakhapatnam",0.142857,2,17.824307,83.356428
76,"Pothinamallayya Palem, Visakhapatnam",0.166667,2,17.799611,83.353089
7,"Anakapalle, Visakhapatnam",0.250000,2,17.689643,83.002363


<b>Observations:</b>

Most of the restaurants are concentrated in the central area of Visakhpatnam city, with the highest number in cluster 2 and moderate number in cluster 1. On the other hand, cluster 0 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new restaurants as there is very little to no competition from existing restaurants. Meanwhile, restaurants in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of restaurants. From another perspective, this also shows that the oversupply of restaurants mostly happened in the central area of the city, with the suburb area still have very few restaurants. Therefore, this project recommends property developers to capitalize on these findings to open new restaurants in neighborhoods in cluster 0 with little or no competition. Property developers with unique selling propositions to stand out from the competition can also open new restaurants in neighborhoods in cluster 0 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 2 which already have high concentration of restaurants and suffering from intense competition.